In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("fra.txt",sep='\t')
data = data.sample(frac=0.1, replace=False)

In [2]:
len(data)

14544

In [3]:
data.head()

,English,French
13111,I'll go with you.,Je vais y aller avec toi.
39371,Don't go without a hat.,N'y va pas sans chapeau.
102422,His sister can't talk to you today.,Sa sœur ne peut pas te parler aujourd'hui.
135337,Crows all but destroyed the farmer's field of ...,Des corneilles ont presque détruit le champ de...
69987,Is there a student discount?,Y a-t-il une remise étudiant ?


In [4]:
data['French'] = data['French'].apply(lambda x:'\t' + x + '\n')

In [5]:
input_chars = set()
output_chars = set()
for index,row in data.iterrows():
    for character in row['English']:
        input_chars.add(character)
    for character in row['French']:
        output_chars.add(character)

In [6]:
input_chars

{' ',
 '!',
 '"',
 '$',
 '%',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é',
 'ú',
 '’'}

In [7]:
output_chars

{'\t',
 '\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 'С',
 '\u2009',
 '’',
 '\u202f'}

In [8]:
input_chars = sorted(input_chars)
output_chars = sorted(output_chars)

In [9]:
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(output_chars)

In [10]:
max_encoder_seq_len = max([len(text) for text in data['English']])
max_decoder_seq_len = max([len(text) for text in data['French']])

In [11]:
max_encoder_seq_len,max_decoder_seq_len

(216, 265)

In [12]:
input_token_index = dict([(char, i) for i, char in enumerate(input_chars)])
output_token_index = dict([(char, i) for i, char in enumerate(output_chars)])

In [13]:
encoder_input_data = np.zeros((len(data['English']), max_encoder_seq_len, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(data['English']), max_decoder_seq_len, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(data['French']), max_decoder_seq_len, num_decoder_tokens),dtype='float32')

In [14]:
input_texts = data['English'].tolist()
output_texts = data['French'].tolist()

In [15]:
for i, (input_text, output_text) in enumerate(zip(input_texts, output_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(output_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, output_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, output_token_index[char]] = 1.


In [16]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
batch_size = 64  # Batch size for training.
epochs = 50  # Number of epochs to train for.
latent_dim = 64  # Latent dimensionality of the encoding space.

Using TensorFlow backend.


In [ ]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

In [ ]:
from keras.layers import Bidirectional

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = Bidirectional(LSTM(latent_dim, return_state=True,return_sequences=True))
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]


In [ ]:






# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [18]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 11635 samples, validate on 2909 samples
Epoch 1/50
11635/11635 [==============================] - 213s 18ms/step - loss: 0.4305 - val_loss: 0.4052
Epoch 2/50
11635/11635 [==============================] - 220s 19ms/step - loss: 0.3675 - val_loss: 0.3526
Epoch 3/50
11635/11635 [==============================] - 220s 19ms/step - loss: 0.3186 - val_loss: 0.3090
Epoch 4/50
11635/11635 [==============================] - 213s 18ms/step - loss: 0.2947 - val_loss: 0.2918
Epoch 5/50
11635/11635 [==============================] - 217s 19ms/step - loss: 0.2817 - val_loss: 0.2820
Epoch 6/50
11635/11635 [==============================] - 225s 19ms/step - loss: 0.2721 - val_loss: 0.2743
Epoch 7/50
11635/11635 [==============================] - 226s 19ms/step - loss: 0.2648 - val_loss: 0.2670
Epoch 8/50
 2560/11635 [=====>........................] - ETA: 2:33 - loss: 0.2608

KeyboardInterrupt: 

In [19]:
from keras.models import load_model

model.save('my_model.h5')

/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:2344: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 64) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [20]:
model = load_model('my_model.h5')

In [21]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [27]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in output_token_index.items())

In [30]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, output_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [31]:
for seq_index in range(10):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: I love you guys.
Decoded sentence: Je ne suis pas de pas de la parie.

-
Input sentence: I'm being promoted.
Decoded sentence: Je ne suis pas de pas de la parie.

-
Input sentence: She did it again.
Decoded sentence: Je ne suis pas de pas de la parie.

-
Input sentence: Don't drink the tap water.
Decoded sentence: Je ne suis pas de pas de la parien.

-
Input sentence: I wish I had seen her.
Decoded sentence: Je ne suis pas de pas de la parie.

-
Input sentence: Doesn't that smell great?
Decoded sentence: Je ne suis pas de pas de la parien.

-
Input sentence: You're invited.
Decoded sentence: Je ne suis pas de pas de la parie.

-
Input sentence: How many computers have you had so far?
Decoded sentence: Je ne suis pas de pas de pas de pas de la parien.

-
Input sentence: I can't believe I'm talking to you about this.
Decoded sentence: Je ne suis pas de pas de pas de pas de pas de la parie.

-
Input sentence: I really wish you would do that.
Decoded sentence: Je ne suis 